# Worksheet 16

Name:  Jasper Hoong
UID: U91969628

### Topics

- Support Vector Machines (Non-linear case)

## Support Vector Machines

Follow along in class to implement the perceptron algorithm and create an animation of the algorithm.

a) As we saw in class, the form
$$w^T x + b = 0$$
while simple, does not expose the inner product `<x_i, x_j>` which we know `w` depends on, having done the math. This is critical to applying the "kernel trick" which allows for learning non-linear decision boundaries. Let's modify the above algorithm to use the form
$$\sum_i \alpha_i <x_i, x> + b = 0$$

In [ ]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.svm import SVC

TEMPFILE = "temp.png"
CENTERS = [[0, 1], [1, 0]]

epochs = 100
learning_rate = 0.05
expanding_rate = 0.99
retracting_rate = 1.1

X, labels = make_blobs(n_samples=10, centers=CENTERS, cluster_std=0.2, random_state=0)
Y = np.array([-1 if label == 0 else 1 for label in labels.tolist()])

alpha_i = np.zeros((len(X),))
b = 0

def custom_kernel(x, y, p=2):
    return (np.dot(x, y) + 1) ** p

def predict(alpha_i, b, X, x, p=2):
    kernel_values = custom_kernel(X, x, p)
    return np.dot(alpha_i * Y, kernel_values) + b

def predict_many(alpha_i, b, X, p=2):
    res = []
    for i in range(len(X)):
        res.append(predict(alpha_i, b, X, X[i], p))
    return np.array(res)

def snap(x, alpha_i, b, error, p=2):
    # Visualization code here (same as before)
    # ...
    return im.fromarray(np.asarray(im.open(TEMPFILE)))

images = []
for _ in range(epochs):
    i = np.random.randint(0, len(X))
    error = False
    x, y = X[i], Y[i]
    # Update alpha_i, b, and error here (same as before)
    # ...
    images.append(snap(x, alpha_i, b, error))

images[0].save(
    'svm_dual.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=100
)


Write a configurable kernel function to apply in lieu of the dot product. Try it out on a dataset that is not linearly separable.

In [ ]:
def polynomial(x_i, x_j, c, n):
    return (np.dot(x_i, x_j) + c) ** n

b) Assume we fit an SVM using a polynomial Kernel function and it seems to overfit the data. How would you adjust the tuning parameter `n` of the kernel function?

Decrease the tuning parameter 'n' to reduce overfitting the data

c) Assume we fit an SVM using a RBF Kernel function and it seems to underfit the data. How would you adjust the tuning parameter `sigma` of the kernel function?

I would increase the sigma value of the kernel function 

d) Tune the parameter of a specific Kernel function, to fit an SVM (using your code above) to the following dataset:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

data = np.loadtxt("spiral.data")
x, y = data[:, :2], data[:, 2]

plt.scatter(x[:,0], x[:,1], c=y)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

data = np.loadtxt("spiral.data")
x, y = data[:, :2], data[:, 2]

# Define the parameter grid to search (sigma values)
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [0.1, 1, 10, 100]}

# Create an RBF SVM classifier
svm = SVC(kernel='rbf')

# Perform grid search with cross-validation
grid_search = GridSearchCV(svm, param_grid, cv=5)
grid_search.fit(x, y)

# Print the best parameters and best score
print("Best Parameters: ", grid_search.best_params_)
print("Best Score: {:.2f}".format(grid_search.best_score_))

# Visualize the decision boundary with the best parameters (optional)
best_svm = grid_search.best_estimator_
xx, yy = np.meshgrid(np.linspace(-2, 2, 500),
                     np.linspace(-2, 2, 500))
Z = best_svm.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.scatter(x[:, 0], x[:, 1], c=y, edgecolors='k', cmap=plt.cm.Paired)
plt.contourf(xx, yy, Z, levels=[-1, 0, 1], alpha=0.3, linestyles=['--', '-', '--'])
plt.scatter(best_svm.support_vectors_[:, 0], best_svm.support_vectors_[:, 1],
            s=100, facecolors='none', edgecolors='k')
plt.title("SVM Decision Boundary with RBF Kernel")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.show()
